- 밑에 함수 선언부터 해야 함

## - OCR

In [ ]:
dirname = 'ocr_sample'
dir_path = 'C:/Users/폴더까지경로/' + dirname + '/'

In [ ]:
ocr_image(dir_path)

## - txtfile 불러오기

In [ ]:
# 폴더 내의 txt 파일을 모두 읽어 리스트로 저장
data_list = []
for txtfile in glob.iglob(dir_path + "*.txt") :
    with open(txtfile, "r", encoding='utf-8') as file :
        read_txt = file.readlines()
        read_txt = np.char.strip(read_txt)
    data_list.append(read_txt)

data_list = np.array(data_list)

# 파싱

In [ ]:
# 파싱한 결과를 DataFrame으로 저장
parse_info = parsing(data_list)
test_info = pd.DataFrame(parse_info, columns=['place', 'date', 'company', 'judgment'])

test_info

In [ ]:
# 검사항목 파싱 결과 DataFrame들을 리스트로 저장
d = test_info['date'].values.tolist()[0]
ocr = [chemi_parsing(dt, d) for dt in data_list]
chem, use = [tb[0] for _,tb in enumerate(ocr)], [tb[1] for _,tb in enumerate(ocr)]

In [ ]:
chem[0]

# 후처리

In [ ]:
# 장소 후처리 결과로 DataFrame 업데이트
place = np.array(test_info.place.tolist())
place = pd.DataFrame({'place':np.vectorize(postp_address)(place)})

test_info.update(place)

test_info

In [ ]:
get_map()

# 함수

In [ ]:
import pandas as pd
import numpy as np
import re
# 파일 읽기
import io
import os
import glob
# 단어 비교
from symspellpy import SymSpell, Verbosity
from symspellpy.helpers import DictIO
from jamo import h2j, j2hcj
# 지도
from google.cloud import vision
import googlemaps
import folium
from folium import plugins
from owslib.wms import WebMapService
from math import sin, cos, sqrt,atan2, radians
import webbrowser

In [ ]:
# 데이터셋 - 전국 주소 / 업체명 / 시료명에 따른 수질검사기준 파일이름 / 오염원 주소
address_dataset = pd.read_csv('address.csv')
col = list(address_dataset.columns)
company_dataset = pd.read_csv('company.csv')
sample_dataset = pd.read_csv('sample.csv')
pollution = pd.read_csv('pollution_address.csv')

In [ ]:
# 입력단어와 후보단어의 해밍거리 계산
def Hamming(inp, suggest, opt):    
    inp_l, sug_l = list(inp), list(suggest)
    # 길이 동일한 경우 바로 계산
    if len(inp) == len(suggest) :
        hamming_val = np.vectorize(Hamming_value)(np.array(inp_l), np.array(sug_l), opt).sum()
        return (hamming_val, 0) if opt is 2 else hamming_val
    
    # 길이 차이 나는 경우 : (나트륨-니트) -> (나트-니트)(트륨-니트) 비교
    hamming = []
    small, big = small_big(inp, suggest)
    dist = big - small
    for i in range(dist+1) :
        if small is len(inp) :
            hamming_val = np.vectorize(Hamming_value)(np.array(inp_l), np.array(sug_l[i:i+small]), opt)
        else :
            hamming_val = np.vectorize(Hamming_value)(np.array(inp_l[i:i+small]), np.array(sug_l), opt)
        
        hamming_val = hamming_val.sum()
        # 길이 차이에 대한 가중치 부여
        if opt == 0 :
            hamming_val += dist * 3
        elif opt == 3 :
            hamming_val += dist
        hamming.append(hamming_val)
        
    # 파싱 중 사용될 경우 인덱스까지 반환
    if opt == 2 :
        hamming = np.array(hamming)
        index = np.where(hamming == hamming.min())
        return hamming.min(), index[0][0]
    
    else :
        hamming.sort()
        return hamming[0]

    
# 각 글자의 자모음 비교
def Hamming_value(input_phone, suggest_phone, opt) :
    value = 0
    # 글자의 자모음 분리
    input_sylla = j2hcj(h2j(input_phone))
    suggest_sylla = j2hcj(h2j(suggest_phone))
    
    if len(input_sylla) == len(suggest_sylla) :
        inp, sug = list(input_sylla), list(suggest_sylla)
        cnt_sylla = len(input_sylla)
    else :
        small, big = small_big(input_sylla, suggest_sylla)
        value += big - small
        
        if small is len(input_sylla) :
            inp, sug = list(input_sylla), list(suggest_sylla)[:small]
        else :
            inp, sug = list(input_sylla)[:small], list(suggest_sylla)
        cnt_sylla = small
        
    inp, sug = np.array(inp), np.array(sug)
    comp = inp == sug
    c = 3 if opt == 3 else 1
    result = np.vectorize(lambda a: -c if a is True else 1)(comp)
    value += result.sum()
        
    return value

# 문자열 길이 비교
def small_big(a, b) :
    small = len(a) < len(b) and len(a) or len(b)
    big = len(a) > len(b) and len(a) or len(b)
    return small, big

In [ ]:
# 후보 중 입력 단어와 가장 비슷한 단어 반환
def find_similar(input_val, cand_list, maxedit=3) :
    maxeditdist = len(input_val)-1 if len(input_val) < maxedit else maxedit
    if maxedit == 4 : # 검사항목 파싱용
        # 해당 단어들이 나오면 비슷한 단어 안찾고 패스
        exceptions = ['기준', '검사', '채수', '의뢰', '시료', '번호', '접수', '적합', '인허가용']
        filt = np.vectorize(lambda a: a in input_val)(exceptions)
        if (input_val.lower() != input_val.upper()) or (filt.any()) :
            return None
    # SymSpell - find candidate
    cand_dict = {cand: 1 for cand in cand_list}
    sym_spell = SymSpell(max_dictionary_edit_distance=maxeditdist)
    dict_stream = DictIO(cand_dict)
    sym_spell.load_dictionary_stream(dict_stream, 0, 1)
    
    suggestions = sym_spell.lookup(input_val, Verbosity.CLOSEST, max_edit_distance=maxeditdist)
    suggestion_list = [suggestion.term for suggestion in suggestions]
    
    list_made = 0
    if not suggestion_list :
        # 검사항목 파싱 시 symspell에서 리스트 안만들어지면 그냥 끝냄
        if maxedit == 4 :
            return None
        
        list_made = 1
        cand = np.array(cand_list)
        inp_v = list(input_val)
        suggest_index = list(map(lambda a: np.where(np.char.find(cand, a)>=0), inp_v))
        suggest_index = np.concatenate(suggest_index, axis=None).tolist()
        suggestion_list = cand[suggest_index]
        suggestion_list = list(set(suggestion_list.tolist()))
    
    if len(suggestion_list) == 1 :
        return suggestion_list[0]
    else :
        if not suggestion_list :
            return None
        if maxedit == 4 and len(suggestion_list) > 5 :
            return None
        hamming_list = [np.vectorize(lambda a: Hamming(input_val, a, list_made))(suggestion_list)]
    
        minv = np.argmin(hamming_list)
        return suggestion_list[minv]

In [ ]:
# substring 찾기
def find_substr(data, substr, ver) :
    exist = np.where(np.core.defchararray.find(data, substr) >= 0)[0]
    if ver is 1 :     # 채수장소 카테고리
        res = exist[0] if data[exist].size > 0 else -1
    elif ver is 2 :   # 시료명
        res = substr if data[exist].size > 0 else None
    return res

# 채수장소

In [ ]:
def isEnter(data, index):
    res = data[index+1] if len(data[index]) < 7 else data[index]
    return res

In [ ]:
def find_place(data_orig):
    data = np.copy(data_orig)
    # 카테고리 찾기
    category = np.array(['채수장소', '시료채취주소', '지하수개발이용시설위치', '지하수개발·이용시설위치'])
    fnd = np.vectorize(lambda a: find_substr(data, a, 1))(category)
    
    if fnd[fnd!=-1].size > 0 :
        place_category = category[fnd != -1][0]
        index = fnd[fnd != -1][0]
        start = np.where(np.core.defchararray.find(data[index], place_category) != -1)[0][0]
        data[index] = data[index][start+len(place_category):]
        content = isEnter(data, index)
        return content
    
    modified_data = np.core.defchararray.replace(data, ' ', '')
    modified_data = np.vectorize(lambda a : ' '+a)(modified_data)
    
    Hamming_place, index = [], []
    for cat in category:
        hammed = np.vectorize(lambda a: Hamming(cat, a, 2))(modified_data)
        line = np.argmin(hammed[0])
        Hamming_place.append(np.min(hammed[0])) 
        index.append([line, hammed[1][line]])
   
    most_similar_category = np.argmin(Hamming_place)
    category, index = category[most_similar_category], index[most_similar_category]
    line, word = index[0], index[1]
    
    categ_last = len(modified_data[line]) - 1 if len(modified_data[line]) <= word+len(category) else word+len(category)-1
    categ_last = modified_data[categ_last]
    last_idx = data[line].find(category_last)

    last_idx += 2 if ((last_idx != len(data[line])-1) and (data[line][last_idx+1] == ' ')) else 0

    data[line] = data[line][last_idx:]
    content = isEnter(data, line)
    
    return content

In [ ]:
def postp_address(placestr) :
    data = address_dataset
    
    input_p = placestr.split()
    not_found_list = []
    
    for num in range(5) :
        if data[col[num]].isnull().values.all() :
            break
            
        not_found = data[data[col[num]].isin([input_p[num]])].empty
        if not_found :
            not_found_list.append(num)
            continue
            
        data = data[data[col[num]] == input_p[num]]
        
    if len(data) != 1 :
        found = find_best(data, input_p, not_found_list)
        data = found[0]
        if found[2] is 1 :
            num = found[1] 
    end_ind = np.where(np.array(input_p)=='의뢰자명')[0][0] if '의뢰자명' in input_p else len(input_p)
    place = ' '.join([data.iloc[0][i] for i in range(num)]) + " " + ' '.join(input_p[num:end_ind])
    return place

In [ ]:
def find_best(df, target, target_num) :
    need_find = target_num
    df_tmp = df
    
    before = np.array(['경남', '경북', '전남', '전북', '충남', '충북'])
    after = np.array(['경상남도', '경상북도', '전라남도', '전라북도', '충청남도', '충청북도'])
    
    if 0 in need_find :
        ind = np.where(before == target[0])[0]
        if ind.size > 0 :
            target[0] = after[ind[0]]
        
    onway = 0
    for i in need_find :
        if df_tmp[col[i]].isnull().values.all() :
            # 전부 nan만 나오게 되면 상세주소 시작함
            onway = 1
            ind = i
            break
                
        candidate = list(set(df_tmp[col[i]].values))
        if len(candidate) == 1 :
            df_tmp = df_tmp[df_tmp[col[i]] == candidate[0]]
        else :
            sim = find_similar(target[i], candidate)
            df_tmp = df_tmp[df_tmp[col[i]] == sim]
        ind = i
            
    res = (df_tmp, ind, onway)
            
    if df_tmp.empty :
        res = find_best(df, target, reversed(target_num))
        
    return res

# 판정 결과

In [ ]:
def find_judgement(data) :
    
    c, i = '적합', '부적합'
    data = np.core.defchararray.replace(data, ' ', '') 
    data = np.delete(data, np.where(data == ''))
    
    c_data = np.vectorize(Hamming)(data, c, 3)
    i_data = np.vectorize(Hamming)(data, i, 3)
        
    judg, H = np.where(c_data < i_data, c, i), np.where(c_data < i_data, c_data, i_data)
    judg_index = np.argmin(H)
    judgement = judg[judg_index]

    return judgement

# 시료명

In [ ]:
def find_sample(data, date):
    #1. 시료명 선택
    sample = sample_dataset['시료명'].values.tolist()
    use = np.vectorize(lambda a: find_substr(data, a, 2))(sample)
    use = use[use != None]
    
    if use.size > 0 :
        use = use[0]
    else :
        hamming = np.vectorize(lambda a: Hamming(a, data, 0))(sample)
        use = sample[np.argmin(hamming)]
        
    #2. dicname 조회
    year, month = int(date[:4]), int(date[6:8])
    
    if year<2018 and month<7 :
        dic_name = sample_dataset[sample_dataset['시료명']== use]['2018 개정 전'].values[0]
    else :
        dic_name = sample_dataset[sample_dataset['시료명']== use]['2018 개정 후'].values[0]

    return use, dic_name+'.txt'

# 채수일자

In [ ]:
def find_date(data):
    # 201X년 XX월 XX일의 문자열 검색
    mtch = np.vectorize(lambda a: re.search('201\d{1}년 \d{2}월 \d{2}일', a))(data)
    mtch = np.delete(mtch, np.where(mtch==None))
    
    if mtch.size > 0 :
        test_date = np.vectorize(lambda a: a.group(0))(mtch)
        final_date = test_date[0] if test_date.size == 1 else cmp_date(test_date)
        return final_date
    
    mtch = np.vectorize(lambda a: re.search('201\d{1}-\d{2}-\d{2}', a))(data)
    mtch = np.delete(mtch, np.where(mtch==None))
    if mtch.size > 0 :
        test_date = np.vectorize(lambda a: a.group(0))(mtch)
        final_date = cmp_date(test_date)
        return final_date
        
    dt = np.vectorize(lambda a: re.search('201\d{1}', a))(data)
    index = np.where(dt != None)
    mtch, mtch_str = dt[index], data[index]
    datestr = np.vectorize(lambda a, b: a[b.start():])(mtch_str, mtch)
    final_date = cmp_date(datestr)
        
    return final_date

In [ ]:
def cmp_date(date_str):
    # 201X년 추출
    year = np.vectorize(lambda a: re.search('201\d{1}', a))(date_str)
    
    # XX월 추출
    month = np.vectorize(lambda a: re.search('\d{2}월', a))(date_str)
    # - XX월 없으면 숫자 XX 찾음
    condm = month == None
    if np.any(condm) :
        month[condm] = np.vectorize(lambda a: re.search("\d{2}", a[4:]))(date_str[condm])
        condm = month == None
    # - 안찾아지면 날짜 없는 것
    if np.all(condm) :
        return None
    if np.any(condm) :
        delete_str = np.where(condm)
        date_str = np.delete(date_str, delete_str)
        year, month = np.delete(year, delete_str), np.delete(month, delete_str)   
        
    # XX일 추출
    day = np.vectorize(lambda a: re.search('\d{2}일', a))(date_str.tolist())
    # - XX일 없으면 숫자 XX 찾음
    condd = day == None
    if np.any(condd) :
        day[condd] = np.vectorize(lambda a, b: re.search("\d{2}", a[4+b.end():]))(date_str[condd], month[condd])
        condd = day == None
    if np.all(condd) :
        return None
    if np.any(condd) :
        delete_str = np.where(condd)
        date_str, year = np.delete(date_str, delete_str), np.delete(year, delete_str)
        month, day = np.delete(month, delete_str), np.delete(day, delete_str)
        
    year_str = np.vectorize(lambda a: a.group(0))(year)    
    month_str = np.vectorize(lambda a: a.group(0))(month)
    day_str = np.vectorize(lambda a: a.group(0))(day)
    
    # 날짜가 1개 이상이면 비교해서 가장 과거의 날짜로 선택
    if date_str.size > 1 :
        cm = cmp(month_str)
        month_str = month_str[cm]
        
        day_str = day_str[cm]
        cd = cmp(day_str)
        day_str = day_str[cd]
        
    test_date = year_str[0] + '년'
    test_date += ' '+month_str[0] if len(month_str[0]) > 2 else ' '+month_str[0]+'월'
    test_date += ' '+day_str[0] if len(day_str[0]) > 2 else ' '+day_str[0]+'일'
        
    return test_date

def cmp(dts) :
    # 각 match객체를 숫자로 변경한 후 최소값이 되는 문자열의 위치 반환
    tonum = np.vectorize(lambda a: int(''.join(filter(str.isdigit, a))))(dts)
    return np.where(tonum == tonum.min())

# 검사항목

In [ ]:
def chemi_parsing(data, date):
    data = np.array(data)
    use, dic_name = find_sample(data, date)
    
    start_index = np.union1d(np.where(data == '준'), np.where(data == '과'))
    start_index = np.union1d(start_index, np.where(data == '검사기준'))
    start_index = np.union1d(start_index, np.where(data == '검사항목'))
    start = start_index[0]+1 if start_index.size > 0 else 0
    
    graph = data[start:]
    
    f = open(dic_name, 'r', encoding='UTF8')
    f_dic = f.readlines()
    f.close()
    
    dic_standard = {}
    for i in f_dic:
        key = i.split()
        dic_standard[key[0]] = key[1]
        
    # [화학물]  [화학물+불검출]
    dic_chemi = list(dic_standard.keys())
    dic_plus = dic_chemi.copy()
    dic_plus.extend(['불검출', '검출'])
    
    # 숫자, '/L' 포함 문자열, 화학물 리스트에 있는 단어 제외한 전체 대상으로 symspell 돌림
    cond1 = np.vectorize(lambda a: a.replace('.', '').replace(',','').isdigit())(graph)
    cond2 = np.char.find(graph, '/L') != -1
    cond3 = np.vectorize(lambda a: a in dic_plus)(graph)
    cond = np.logical_or(cond1, cond2)
    cond = np.logical_or(cond, cond3)
    
    need_change = graph[cond == False]
    ind_tmp = np.where(cond == False)
    graph[cond == False] = np.vectorize(lambda a: find_similar(a.replace('.',','), dic_plus, 4))(need_change)
    
    chemi = graph[np.vectorize(lambda a: a in dic_chemi)(graph)]
    chem, ind = np.unique(chemi, return_index=True)
    chemi = chem[ind.argsort()]
    
    result_index = np.union1d(np.where(cond1 == True), np.where(graph == '불검출'))
    result_index = np.union1d(result_index, np.where(graph == '없음'))
    result = graph[result_index]
    
    standard = np.vectorize(lambda a: dic_standard[a])(chemi)    
    # - 결과값이 덜 인식된 경우 ; NaN으로 채움
    if chemi.shape > result.shape :
        nans = np.empty((chemi.shape[0] - result.shape[0],))
        nans.fill(np.nan)
        result = np.concatenate((result, nans))
    # - 결과값이 더 인식된 경우 ; 잘라냄
    elif chemi.shape < result.shape :
        result = result[:chemi.shape[0]]
        
    ocr = pd.DataFrame(np.array([chemi, standard, result]).T, columns=['검사항목', '기준', '결과'])
    
    return ocr, use

In [ ]:
def parsing(data_list):

    return_list = []
    for data in data_list :
        place = find_place(data)
        place = place.replace('|', '')
        date = find_date(data)
        test_result = find_judgement(data)
        
        # find company (업체명)
        company_data = company_dataset
        pattern = company_data['패턴'].values.tolist()
        
        mtch = [patt for patt in pattern for txt in data if patt in txt.replace(' ', '')]
        found_pattern = list(set(mtch))
        
        company = company_data[company_data['패턴'].isin(found_pattern)]
        company = list(set(company['업체명'].values))[0]
        
        return_list.append([place, date, company, test_result])

    return return_list

# 수행


In [ ]:
def ocr_image(dir_path):
    json_path = 'C:/ocr json 파일까지 경로'
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_path
    client = vision.ImageAnnotatorClient()

    # 폴더명 입력시 폴더 내의 이미지 이름과 같은 이름으로 txt 파일 생성
    #dirname = input("샘플 이미지가 들어있는 폴더명을 입력하세요 : ")

    for file in glob.glob(dir_path + "*.jpg") :
        img_name = (os.path.basename(file))
        txt_name = img_name.replace('.jpg', '.txt')

        f = open(dir_path+txt_name, 'w', encoding = 'utf-8')

        file_name = os.path.abspath(file)

        with io.open(file_name, 'rb') as image_file :
            content = image_file.read()

        image = vision.Image(content=content)
        response = client.document_text_detection(image = image)

        f.write(response.text_annotations[0].description)
        f.close()

# 지도

In [ ]:
g_maps = googlemaps.Client(key='google maps key값')
def getLatLng(address):
    result = ""
    geocode_result = g_maps.geocode(address)
    
    lat = geocode_result[0]["geometry"]["location"]["lat"]
    lng = geocode_result[0]["geometry"]["location"]["lng"]
    
    return [lat, lng]

# 두 장소 사이 거리 구하기 (단위 km)
def get_distance(lat1, lng1, lat2, lng2):
    R = 6373.0
    
    lat1, lng1 = radians(lat1), radians(lng1)
    lat2, lng2 = radians(lat2), radians(lng2)
    
    dlon = lng2 - lng1
    dlat = lat2 - lat1
    
    a = sin(dlat/2)**2+cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2*atan2(sqrt(a), sqrt(1-a))
    
    return R*c

In [ ]:
def get_map():
    address = [getLatLng(place) for place in test_info.place.tolist()]
    cent = [35.95, 128.25]
    distance = float(input("반경 입력 (단위:m) : "))
    m = folium.Map(location = cent, zoom_start = 8)

    classes = ("table table-striped table-hover table-condensed table-responsive")
    colours = {'도축업장': 'red', '축산농장': '#ff751a', '폐기물처리장': '#ffcc00',  '골프장': '#33cc33', '주유소': '#33ccff', 
                '공공하수처리시설': 'blue', '지하수정화시설': '#b366ff', '기타수질오염원설치시설': '#ff4dd2', '공장': 'black'}
    legend_html = """<div style="position:fixed; bottom:10px; left:10px; width:155px; height:175px; 
                        border:2px solid black; z-index:9999; background-color:white; opacity: .75;
                        font-size:12px;">&nbsp;<b>"""+'오염원'+""":</b><br>"""

    #setting subgroups
    con = folium.FeatureGroup(name='적합')
    incon = folium.FeatureGroup(name='부적합')
    pol = plugins.FeatureGroupSubGroup(incon, name='오염원', show=False)
    m.add_child(con)
    m.add_child(incon)
    m.add_child(pol)

    # colour setting legend
    for key, value in colours.items():
        legend_html = legend_html+"""&nbsp;<i class="fa fa-circle 
                                    fa-1x" style="color:"""+value+"""">
                                    </i>&nbsp;"""+key+"""<br>"""
    legend_html += """</div>"""
    m.get_root().html.add_child(folium.Element(legend_html))

    for i, addr in enumerate(address):
        html = f"""<p><strong>채수장소 : </strong>{test_info.place[i]}</br>
                <strong>채수일자 : </strong>{test_info.date[i]}</br>
                <strong>업체명 : </strong>{test_info.company[i]}</br>
                <strong>판정 : </strong>{test_info.judgment[i]}</br>
                <strong>시료명 : </strong>{use[i]}</p>"""

        popup = html + chem[i].to_html(classes=classes)
        popup = folium.Popup(popup, max_width=500)

        if test_info.judgment[i] == '적합':
            folium.Marker(addr, popup=popup,tooltip="결과확인", icon=folium.Icon(color='blue', icon='ok')).add_to(con)
        else:
            folium.Marker(addr, popup=popup,tooltip="결과확인", icon=folium.Icon(color='red', icon='remove')).add_to(incon)
            folium.Circle(addr, radius=3000, popup=None, color="black").add_to(pol)
            for j in pollution.index :
                loc = pollution.loc[j, '좌표'].strip('][').split(', ')
                ch0 = float(loc[0])   #좌표가 string형이라 float형으로 바꿔준다.
                ch1 = float(loc[1])

                dist = get_distance(addr[0], addr[1], ch0, ch1)

                if dist <= (distance/1000) : #단위 km
                    colour = colours[pollution.loc[j, '분류']]
                    icon = plugins.BeautifyIcon(icon="exclamation-sign", text_color=colour)
                    folium.Circle(
                        location = (loc[0], loc[1]),
                        tooltip = pollution.loc[j, '사업장명'],
                        radius=40, color=colour, fill_color=colour, fill_opacity=1.0
                        ).add_to(pol)


    # 지하수 유동방향 지도
    url="http://www.gims.go.kr/api/wms?KEY=rwKBcuRhkThP9tNcW%2F6NA2bMWnTefj8BXA8C1YAkEqBE9%2FzNldesNIA7Szu7AY1R"
    webms = WebMapService(url, version='1.3.0')
    layer = '36'
    wms = webms.contents[layer]
    lon = (wms.boundingBox[0] + wms.boundingBox[2]) / 2.0
    lat = (wms.boundingBox[1] + wms.boundingBox[3]) / 2.0

    m_wms = folium.raster_layers.WmsTileLayer(url=url, fmt="image/png", name=wms.title, transparent=True, layers=layer, 
                                              overlay=True, control=True, show=False)
    m_wms.add_to(m)
    
    folium.LayerControl(collapse=False).add_to(m)
    
    result_map = "map.html"
    m.save(result_map)
    webbrowser.open(result_map, new=2)
    
    return m